<a href="https://colab.research.google.com/github/jung0228/ML/blob/main/%EC%98%81%ED%99%94%EB%A6%AC%EB%B7%B0_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***선행작업***

In [ ]:
!pip install konlpy
!pip install collections
!pip install wordcloud
!pip install sweetviz


ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections
     |████████████████████████████████| 15.1 MB 4.5 MB/s 


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import sys
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import sklearn.metrics as metrics
import math
import sweetviz as sv
import missingno as msno
# 다운받은 csv를 pandas의 DataFrame 형식으로 불러옵니다.
data = pd.read_csv('train.csv')

data[20:100] # 데이터를 확인합니다.

,id,document,label
20,21,"처절한 영화, 슬픈 영화, 씁쓸-한 영화...",1
21,22,처음부터 끝까지 유치 하기 그지 없다....난해한 영화,0
22,23,전쟁? 아이들의 목소리가 참 거슬린다.,0
23,24,멋진 상상력! 아름다운 영화. 여배우 소이의 재발견!,1
24,25,그 시대엔 이 영화가 최고였음,1
...,...,...,...
95,96,표 값과 시간이 아까웠던 영화,0
96,97,쩌는 시나리오 쩌는 연기 환상의 소피아 결론은 제이미 벨 이 축복받은 놈,1
97,98,너무 기분좋게봣습니다~ 적극 추천!!,1
98,99,왜 그랬니? 피트 출현할 영화가 그렇게 없디?,0


# ***데이터 분석하기***

## ***그냥 둘러보기***

In [ ]:
df_train = data.copy()
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5000 non-null   int64 
 1   document  5000 non-null   object
 2   label     5000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 117.3+ KB


In [ ]:
#결측값은 없는지 확인하기
for col in df_train.columns:
    msg = 'column: {:>20} \t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

column:                   id 	 Percent of NaN value: 0.00%
column:             document 	 Percent of NaN value: 0.00%
column:                label 	 Percent of NaN value: 0.00%


## ***(문장)끼리 비교해보기***

In [ ]:
val_count = data['label'].value_counts() # 유니크값의 개수를 확인합니다.
print('라벨 0인 리뷰 개수:', val_count[0])
print('라벨 1인 리뷰 개수:', val_count[1])

라벨 0인 리뷰 개수: 2564
라벨 1인 리뷰 개수: 2436


In [ ]:
len_1 = 0
len_0 = 0
for i in range(len(df_train)):
  str = df_train['document'][i]
  if df_train['label'][i] == 1:
    len_1 += len(str)
  if df_train['label'][i] == 0:
    len_0 += len(str)
  
print(f"긍정 평가 평균 길이 : {len_1/val_count[1]}\n부정 평가 평균 길이 : {len_0/val_count[0]}")

긍정 평가 평균 길이 : 27.188013136288998
부정 평가 평균 길이 : 27.085023400936038


##***(단어)의 통계적인 비율 살펴보기***

In [ ]:
bad_word = {}
bad_num = 0

for i in range(5000):
  if df_train['label'][i] == 0:
    text = df_train['document'][i]
    okt = Okt()
    noun = okt.nouns(text)
    #print(noun)
    for i in noun:
      if i not in bad_word:
        bad_word[i] = 1
        bad_num += 1
      else:
        bad_word[i] += 1
        bad_num += 1

# lambda x : x[1] 
d1 = sorted(bad_word.items(), key=lambda x: x[1], reverse=True) 
print(d1)
print(f"전체 악평 단어 개수 : {bad_num}")


[('영화', 754), ('이', 143), ('진짜', 133), ('점', 130), ('평점', 119), ('왜', 116), ('재미', 105), ('정말', 104), ('스토리', 102), ('최악', 98), ('시간', 98), ('내용', 90), ('것', 89), ('뭐', 87), ('감독', 79), ('거', 78), ('내', 77), ('그냥', 77), ('돈', 75), ('연기', 73), ('쓰레기', 70), ('배우', 69), ('좀', 67), ('말', 65), ('이건', 61), ('걸', 54), ('끝', 53), ('별로', 53), ('보고', 52), ('더', 48), ('액션', 43), ('나', 43), ('그', 41), ('하나', 41), ('사람', 41), ('생각', 40), ('완전', 40), ('정도', 39), ('개', 38), ('드라마', 37), ('볼', 37), ('연출', 37), ('수준', 37), ('실망', 36), ('듯', 35), ('못', 34), ('이해', 34), ('점도', 33), ('처음', 32), ('소재', 32), ('원작', 31), ('무슨', 31), ('수', 30), ('뿐', 29), ('때', 29), ('코미디', 29), ('임', 29), ('결말', 28), ('때문', 28), ('주인공', 27), ('전혀', 26), ('느낌', 26), ('감동', 26), ('게', 26), ('이야기', 26), ('졸작', 25), ('움', 25), ('차라리', 24), ('별', 24), ('뭔가', 24), ('급', 23), ('위', 22), ('안', 22), ('편', 22), ('애', 21), ('작품', 21), ('보기', 20), ('공포', 20), ('긴장감', 20), ('노잼', 20), ('지루함', 20), ('스릴러', 20), ('시나리오', 20), ('저', 20), ('

In [ ]:
good_word = {}
good_num = 0
for i in range(5000):
  if df_train['label'][i] == 1:
    text = df_train['document'][i]
    okt = Okt()
    noun = okt.nouns(text)
    #print(noun)
    for i in noun:
      if i not in good_word:
        good_word[i] = 1
        good_num += 1
      else:
        good_word[i] += 1
        good_num += 1
# lambda x : x[1] 
d2 = sorted(good_word.items(), key=lambda x: x[1], reverse=True) 
print(d2)
print(f"전체 호평 단어 개수 : {good_num}")

[('영화', 769), ('정말', 216), ('최고', 214), ('진짜', 136), ('감동', 135), ('연기', 111), ('드라마', 90), ('보고', 89), ('때', 83), ('이', 79), ('사랑', 74), ('다시', 72), ('평점', 67), ('점', 67), ('것', 66), ('생각', 65), ('말', 63), ('명작', 61), ('내', 58), ('배우', 58), ('수', 55), ('더', 55), ('사람', 55), ('왜', 55), ('완전', 54), ('마지막', 52), ('가슴', 51), ('볼', 51), ('이야기', 50), ('그', 49), ('작품', 48), ('재미', 47), ('또', 46), ('꼭', 45), ('중', 45), ('역시', 44), ('거', 43), ('인생', 41), ('나', 40), ('지금', 40), ('액션', 36), ('매력', 35), ('마음', 34), ('여운', 34), ('기억', 33), ('하나', 33), ('장면', 33), ('음악', 31), ('감독', 31), ('추천', 29), ('굿', 29), ('난', 29), ('스토리', 28), ('기대', 28), ('계속', 27), ('가장', 27), ('번', 27), ('임', 26), ('강추', 26), ('정도', 26), ('최고다', 26), ('우리', 25), ('아이', 25), ('한번', 25), ('나름', 25), ('속', 25), ('처음', 25), ('삶', 23), ('짱', 23), ('느낌', 23), ('이해', 23), ('모두', 23), ('요', 23), ('눈물', 22), ('내용', 22), ('잼', 22), ('대박', 22), ('수작', 22), ('그냥', 21), ('현실', 21), ('표현', 20), ('주인공', 20), ('인간', 20), ('시리즈', 19), ('추

In [ ]:
print(f"부정 단어 가짓수 : {len(d1)}")
print(f"긍정 단어 가짓수 : {len(d2)}")

부정 단어 가짓수 : 3227
긍정 단어 가짓수 : 3110


In [ ]:
def persent(n, word_num, d):
  sum = 0
  for i in range(n):
    sum += d[i][1]/word_num
  print(f"단어 {n}개 사용했을 때 비율 : {round(sum*100, 2)} %")

In [ ]:
print("부정 단어")
for i in range(500, 2000, 200):
  persent(i, bad_num, d1)
print("\n긍정단어")
for i in range(500, 2000, 200):
  persent(i, good_num, d2)

부정 단어
단어 500개 사용했을 때 비율 : 68.23 %
단어 700개 사용했을 때 비율 : 73.59 %
단어 900개 사용했을 때 비율 : 77.42 %
단어 1100개 사용했을 때 비율 : 80.79 %
단어 1300개 사용했을 때 비율 : 83.74 %
단어 1500개 사용했을 때 비율 : 85.43 %
단어 1700개 사용했을 때 비율 : 87.12 %
단어 1900개 사용했을 때 비율 : 88.81 %

긍정단어
단어 500개 사용했을 때 비율 : 68.85 %
단어 700개 사용했을 때 비율 : 74.08 %
단어 900개 사용했을 때 비율 : 77.72 %
단어 1100개 사용했을 때 비율 : 81.36 %
단어 1300개 사용했을 때 비율 : 83.54 %
단어 1500개 사용했을 때 비율 : 85.36 %
단어 1700개 사용했을 때 비율 : 87.18 %
단어 1900개 사용했을 때 비율 : 88.99 %


In [ ]:
#겹치는 단어는 얼마나 있을까?
for i in range(1000):
  for j in range(1000):


### ***단어 목록 살펴보기 (수위가 높은 단어들도 많이보인다)***

In [ ]:
for i in d1:
  print(i[0])

영화
이
진짜
점
평점
왜
재미
정말
스토리
최악
시간
내용
것
뭐
감독
거
내
그냥
돈
연기
쓰레기
배우
좀
말
이건
걸
끝
별로
보고
더
액션
나
그
하나
사람
생각
완전
정도
개
드라마
볼
연출
수준
실망
듯
못
이해
점도
처음
소재
원작
무슨
수
뿐
때
코미디
임
결말
때문
주인공
전혀
느낌
감동
게
이야기
졸작
움
차라리
별
뭔가
급
위
안
편
애
작품
보기
공포
긴장감
노잼
지루함
스릴러
시나리오
저
기억
개봉
건
난
중
류
용
후
뭘
포스터
전개
장면
공포영화
알바
줄
머
캐릭터
점수
여자
시리즈
자체
대체
욕
반전
도
이영화
보지
제목
장르
코
일본
마지막
발연기
도대체
한국
음
진심
설정
막장
기대
킬링타임
예술
좀비
코믹
비디오
함
이유
이제
누가
이상
접
삼류
물
미국
지루
대한
기분
유치
의도
의미
내내
감
웃음
해
속
건가
머리
영화관
초반
공감
가장
어이
아이
후회
관객
더빙
갈수록
다큐
이하
가지
표현
극장
영화로
술
짜증
인생
애니
지금
만화
출연
한국영
다른
식
점점
평가
알
자기
캐스팅
엉망
산
웃기
잼
아주
어디
한마디
계속
꽝
어색
작가
장난
연기력
데
다시
요
구성
놈
반개
역시
네이버
이름
억지
자
각본
낭비
제일
그래픽
전편
씬
간다
여기
위해
누구
질
개연
영
몰입
시작
영상
성룡
나름
또
발
로
전부
제대로
젠
모든
한번
작
책
집중
엔딩
배
점준
프랑스
제발
편집
일
모
눈
부족
잠
의
그다지
전형
선택
그닥
우리
비
음악
해도
현실
부분
전
줄거리
예전
열
오
핵
비현실적
시대
얼굴
척
소리
망작
구리
리메이크
낚
질질
시사회
주연
얘기
무엇
절대
별거
흥행
맛
첨
팬
게임
주기
정신
마무리
나머지
만
남
주기도
손발
포장
중간
배경
미화
지도
당시
사랑
낚였다
로써
분명
판
속편
비교
마
예고편
평
만들기
인물
표절
초딩
마음
화가
암
남자
추천
은
조금
극치
화보
다가
영화인
하품
앞
뒤
분
기세
사실
최고
자극
번
볼일
불가
모두
명
티비
전달
란
글
창
제작
뻔
니
봣
스타
개판
졸라
아무
스릴
명작
전작

In [ ]:
for i in d2:
  print(i[0])

영화
정말
최고
진짜
감동
연기
드라마
보고
때
이
사랑
다시
평점
점
것
생각
말
명작
내
배우
수
더
사람
왜
완전
마지막
가슴
볼
이야기
그
작품
재미
또
꼭
중
역시
거
인생
나
지금
액션
매력
마음
여운
기억
하나
장면
음악
감독
추천
굿
난
스토리
기대
계속
가장
번
임
강추
정도
최고다
우리
아이
한번
나름
속
처음
삶
짱
느낌
이해
모두
요
눈물
내용
잼
대박
수작
그냥
현실
표현
주인공
인간
시리즈
추억
편
좀
시간
제일
영상
연출
이상
안
소재
만
알
움
듯
걸작
뭐
남자
전
충격
당시
여자
결말
끝
한국
연기력
보기
때문
저
기분
아주
대한
웃기
배경
엔딩
모습
명
다른
흥미진진
적
함
한국영
시대
이제
이영화
개인
분위기
교훈
몰입
꽤
감
오늘
자체
줄
의
옛날
도
눈
애니메이션
후
대해
만점
조금
가족
시즌
몇번
반전
작
여배우
재
그녀
애니
코미디
대사
제목
소름
코믹
걸
공감
내내
이건
의미
화이팅
세계
나이
개
데
시절
시청률
시작
초딩
상미
후회
분
슬픔
무조건
존재
애
요즘
영화인
로
이후
만화
년
동안
벌써
일본
자
원작
남
성장
캐릭터
인상
예술
재밋
노래
전쟁
씬
기도
은근
별
건
봄
팬
만하
역대
제
못
앞
사실
꿀잼
구성
맘
스타일
티비
실화
발견
스릴
진심
짱짱
집
기
뭔가
결코
울
모든
그대로
힘
순간
보
평생
바로
다운
친구
한편
해
약간
너
오빠
강
디즈니
날
극장
손
당신
필요
비극
볼때
몰입도
우리나라
별로
절대
두
명연기
달달
환상
그때
밋
좀비
와우
를
소녀
누가
초반
담
살
코
미국
전편
이제야
완성
파리
멋
언제
개봉
소장
로맨틱
냄새
관
욕망
니
으리
대안
올해
얼마나
한마디
점수
이연걸
짐
긴장감
어른
기적
상상력
신분
인물
심장
압권
꿈
닥치고
웃
더욱
감명
스
폴
집중
묘
보지
무엇
천재
전개
점도
미가
잘만
봣
선생님
전혀
발
그림
갈수록
스릴러
재밋다
동화
신인
명품
대작
거의
대한민국
매우
평론가
다소
특유
서로
라면
준
멜로
역
구
선
여
최근
고전
면서
여주
가끔
설명
캐리
이유
영화로

# ***예측해보기***
